# Building Solutions with LLMs and Retrival Augmented Geenration (RAG): Introduction

Welcome to this AI workshop! In this workshop we will get deeper into AI, RAG, chatbots, embeddings, evaluations and more.
But first lets get used to this notebook enviroment.

### Getting started with Jupyter Notebooks
Notebooks are one of the most imporant tools in teh toolbelt of a data scientist of ML engineer.
They are great for interacting with code and visualizing results.
This file you are interacting with is a jupyter notebook. In Jupter you can have cells of either text or code.
Cells with code can be run by pressing ctrl+enter.



# Task 1
1. Run the cell bellow a couple of times to get used to the notebook behaviour.
2. Restart the kernel (Restart) button above and run again this code to see the difference.

In [ ]:
# define a if it does not exist
if  'a' not in globals():
    a = 42

a = a+1

# press ctrl+enter to run the cell, do that multiple times
# notice how the variable `a` is persistent in the cell
# you can run full programs in a notebook.

a


## LLMs
Large Language Models (LLMs) are a type of machine learning models designed to understand and generate human language. They are trained on massive datasets of text to predict and generate language based on given prompts, learning patterns, structures, and relationships in text to produce human-like responses. They can be used to generate text, answer questions, and more.

### MistralAI

For this workshop we wil use [Mistral](https://mistral.ai/) langauge models, which are similar in concept to OpenAI's ChatGPT and Anthropic's Claude. If your installation finalized correctly you should have mistral already installed. 
The command below checks if mistral is installed.


In [ ]:
!pip show mistralai

If for some reason mistral did not installed successfully you can get it (and all other dependencies of this workshop by installing it directly with the command below).

In [ ]:
# run this command only if mistral is not installed 
!pip install -r ../requirements.txt
!pip install -e ../

### Getting access to Mistral LLMs

LLMs are usually big and use a lot of computer resources the most common way to use them is to run them in teh cloud via network calls. For the sake of this workshop we will use the cloud version as well so we dont need to download big models.
If you are interested in running llms locally check [ollama](https://github.com/ollama/ollama), arguably the most advanced project that does it.


#### API key
The second  step is to get a Mistral api key. You can find some APIs keys we prepared for this workshop in this [sheet](https://docs.google.com/spreadsheets/d/1ZwTpkG6OOuVrOx8nzPmgai_7Hwpo8Kun7yZrOmg_5K4/edit?gid=0#gid=0). Get the key (please write your name next to it in the sheet such that people know it is taken) and write it to the .env file using the command


# Task 2

Use the cell below to write your api key into the file so you can use mistral.
We write the variable into a file named .env. .env files a are a standard in python to load information like api keys and passwords that should not stay with the code for security reasons.

In [ ]:
# write file .env

mistral_api_key = 'UXBen5CqwpPmPYloWuW2uOhaPjxC8irA'
with open('../.env', 'w') as f:
    f.write(f'MISTRAL_API_KEY="{mistral_api_key}"')

# make sure to delete teh API key from this cell before commiting the file so you dont save your key in the repo which is a security risk.

# Checking if it worked

Run the command below to check if writing the key worked.  It will trigger an error if it did not.  You might need to restart the kernel if it does not work.

In [ ]:
# we now reload the the configuration file and it should show your key
from dotenv import load_dotenv, find_dotenv
env_file = find_dotenv()
print(f"Loading environment variables from {env_file}")
load_dotenv(env_file)

import os
env = os.getenv('MISTRAL_API_KEY')

# you should see your key here not
print("The key is: ", env)

if not env:
    raise ValueError("The API key is not set. Please set the MISTRAL_API_KEY environment variable.")


## Running mistral
Let's run the code below to import Mistral and initialize the Mistral client: 

In [ ]:
import os
from mistralai import Mistral

# Retrieve the Mistral API key from the environment variables
mistral_api_key = os.getenv('MISTRAL_API_KEY')

# Initialize the Mistral client with the API key
mistral_client = Mistral(api_key=mistral_api_key)

# The model below is the specific model we want to use
model_name = "mistral-small-latest"

# The code below defines a function `call_mistral_model` that sends a message to a Mistral model and returns the model's response text.
response = mistral_client.chat.complete(
    model = model_name,
    messages = [
        {
            "role": "user",
            "content": "hello! What is your name?",
        }
        ]
    )
# Extract only the text from the response
response_text = response.choices[0].message.content
print(response_text)

## not how the result resembles natural language. Its the exact same concept as chatgpt.
## feel free to play around with the prompt and see how the results change.

Instead of having all of this complex code to call the llm, going forward, we can move this complexity to a python script.
This is what we are doing from the code below on. We are replacing our calls to mistral api to calls to LargeLanguageModel the language model class.

Rate limiting is a mechanism implemented by APIs to control the number of requests a user can make in a given time frame. This is done to prevent abuse so 1 player dont make the whole capacity used and slow.  However, rate limiting can be annoying because it can interrupt your workflow and force you to wait before making more requests.

To make things easier, we've implemented a rate limit error controller in LargeLanguageModel class (see usage example below) that automatically adds sleep intervals between requests to avoid exceeding the rate limit. We will use the LargeLanguageModel class moving forward to make calls to Mistral AI. This class has the same logic as the examples we showed above but includes a mechanism to counter the rate limiting issue. 

Have a look into the class by ctrl+ clicking in the **LargeLanguageModel* name to jump directly to its implementation.

In [ ]:
from chat_solution.llm import LargeLanguageModel

# Initialize an instance of the LargeLanguageModel class
llm = LargeLanguageModel()
# Make a call to Mistral using the LargeLanguageModel class
response = llm.call("hello! What is your name?")

print(response)

## Exploring the LLM
Now that we have seen how to make a basic call to the Mistral model using the [LargeLanguageModel](../chat_solution/llm.py) class, let's try some more prompts to see how the model responds to different types of queries.

# Exploring and Modifying Prompts
Below are some example use cases of how to use an LLM such as Mistral. Play around with the prompts and see the results. Modify the prompts to see how the model's responses change. This will help you understand how to craft effective prompts and get the desired output from the model.

Try to:
- Ask different types of questions
- Change the text for summarization or extraction (see examples 2 and 3 below)
- Alter the style of the response

#### Example 1: Asking for Information

In [ ]:
# Example: Asking for information
prompt = "Can you tell me about coding school 42Berlin?"
response = llm.call(prompt)
print(response)

#### Example 2: Summarizing a Given Text

In [ ]:
# Change the text into something else to see the results
text_to_summarize = (
    """
    42Berlin is a non-profit coding school offering software engineering education completely tuition free. 
    By making tech education more accessible and inclusive, they empower the next generation of coders.
    Founded in 2021 and based in central Neukölln, we train our students up to the equivalent of Master’s level 
    and implement peer-learning methodologies that give autonomy to each student.
    """

)
prompt = f"Summarize the following text in one brief sentence: {text_to_summarize}"
response = llm.call(prompt)
print(response)

#### Example 3: Extracting Information from a Given Text

In [ ]:
# Change the text into something else to see the results
text_to_extract_from = (
    """
    42Berlin is a non-profit coding school offering software engineering education completely tuition free. 
    By making tech education more accessible and inclusive, they empower the next generation of coders.
    Founded in 2021 and based in central Neukölln, we train our students up to the equivalent of Master’s level 
    and implement peer-learning methodologies that give autonomy to each student.
    """

)
prompt = f"Extract the year 42Berlin was founded from the following text: {text_to_extract_from}"
response = llm.call(prompt)
print(response)


# Task 3: Doing maths -> When will the LLM make a mistake?

LLMs are trained on text language so they are not necessarily good with maths. We can be confident that it will fail to produce the correct result if you ask a question that is complex enough. On the notebook below we increase the complexity of the task at every run. Run it until you see it diverging.

What was te number of iterations necessary to make it break?

In [13]:
if 'a' not in globals() or 'b' not in globals():
    a = 1
    b = 1
a = a + 1
b = b + 1

## ** in python is the power operator meaning: a to the power of b
print("Correct result: ", a**b)
response = llm.call(f"what is the results of {a} ** {b}")
print("Mistral result: ", response)

Correct result:  10000000000
Mistral result:  The result of \(10^{10}\) is 10 raised to the power of 10, which equals 10,000,000,000.

Here's the calculation:

\[10^{10} = 10 \times 10 \times 10 \times 10 \times 10 \times 10 \times 10 \times 10 \times 10 \times 10 = 10,000,000,000\]


As you probably noted, mistral does not only return the results, it also explains it step by step. For example:

```
The result of 4 raised to the power of 4 (4 ** 4) is calculated as:
4 * 4 * 4 * 4 = 256
```

Mistral and other language models deliberatelly add this longer explanations as they help the model commit less mistakes. This pattern is also known as chain of thought reasonsing. And is one of the most robust ways to improve LLM's responses. When you write prompts think about how to explain the steps of the reasonsing to improve the performance of your propmts.


## Hallucination
LLMs sometimes generate responses that are plausible-sounding but factually incorrect or nonsensical. This phenomenon is known as "hallucination". 
Hallucination can occur because the model generates text based on patterns in the training data rather than actual knowledge or retrieval of relevant information.
LLMs will produce the most likelly words that they would find in similar text, they have no clue about fact vs fiction. They can confidently produce writing about things they have no clue about.


### Exercise 2: Demonstrating Hallucination

In this exercise, we will ask the model a question that it might to hallucinate an answer for, showing the limitations of relying solely on language generation without retrieval.

Try running the command below a few times in a row and see how the response by the LLM changes.



In [15]:
# Ask a question likely to cause hallucination
prompt = "What was the name of the workshop launched by the MLOps Community Berlin in collaboration with Girls in Tech?"
response = llm.call(prompt)
print("Response likely to hallucinate:\n")
print(response)

Response likely to hallucinate:

The workshop launched by the MLOps Community Berlin in collaboration with Girls in Tech was named "MLOps for Beginners." This workshop aimed to introduce participants to the fundamentals of MLOps (Machine Learning Operations) and provide them with practical skills and knowledge to implement MLOps in their projects.


Now, let's move on to the next section on Retrieval-Augmented Generation.

# Retrieval-Augmented Generation (RAG)

Retrieval-Augmented Generation (RAG) is a technique that incorporates external information to a LLM to generate better responses. 

In RAG, a retrieval component searches and retrieves relevant information from a knowledge base or external documents, and a generation component uses this information to generate responses.
This approach allows the model to access up-to-date information and provide more detailed and accurate answers.

### Exercise 4: Simple RAG

In this exercise, we will demonstrate a simple example of how to use Retrieval-Augmented Generation. We will use a predefined set of documents, retrieve relevant information based on a query, and then generate a response using the retrieved information.

Run the code below.

In [16]:
def create_rag_prompt(message: str, context: str):
    """
    Message is the question that the user is asking.
    Context is the information that we want to use to answer the question.
    """
    return f"""Answer the question only using the provided content.

        Context: {context}

        User Question: {message}

        Be helpful and friendly. If the information cannot be found respond with "I don't know"
        """  

By running the code in the cell below, you can compare how our LLM responses differ by the information that you provided.

In [19]:


message = "What was the name of the workshop launched by the MLOps Community Berlin in collaboration with Girls in Tech?"


generic_response = llm.call(message)
print(f"GENERIC RESPONSE:\n {generic_response}")


# The workshop the MLOps Community hosted together with Girls in Tech Germany was called "AI Launchpad: Building Your First Ml Pipeline" or simply "Building Your First ML Pipeline"
# We copy paste the info from our Eventbrite event page from the previous workshop and use this as context for the model to retrieve the right info from
context = """Title: AI Launchpad - Building Your First ML Pipeline: 
On Wednesday, June 5th, 2024, the MLOps Community Berlin in collaboration with Girls in Tech Germany hosted an interactive workshop for beginners who want to kick start their career in AI/ML. 
The workshop starts at 18.00h at 42Berlin. 

🔍 Why Attend?

Gain hands-on experience building your first ML pipeline in an agile way
Apply the fundamentals of statistical modeling and basic Python
Opportunities to improve your portfolio 
Connect with ML professionals at different levels of seniority


✨ The Agenda: 

6:00 pm - Arrive & Pizza 
6:30 pm -  Introduction MLOps and GiT
6:45 pm - Workshop Introduction
7:30 pm - Break
7:45 pm - Workshop
9:45 pm - Networking


🎉 Highlights:

Food and drinks provided
Engaging discussions and networking opportunities
Bring your laptop and get ready to learn!


💼 Who Should Attend?

Individuals starting their career in Machine Learning or Artificial Intelligence
Those looking to transition into the field of AI/ML
Anyone interested in contributing to and learning from the ML community
Don't miss out on this chance to gain practical AI/ML skills while expanding your professional network! 
"""
rag_prompt = create_rag_prompt(message=message, context=context)
rag_response = llm.call(rag_prompt)

print("-" * 30)
print(f"RAG RESPONSE:\n {rag_response}")

GENERIC RESPONSE:
 The workshop launched by the MLOps Community Berlin in collaboration with Girls in Tech was named "MLOps for Women." This workshop aimed to empower women in the field of machine learning operations (MLOps) by providing them with practical skills, knowledge, and networking opportunities.
------------------------------
RAG RESPONSE:
 The name of the workshop launched by the MLOps Community Berlin in collaboration with Girls in Tech is "AI Launchpad - Building Your First ML Pipeline."


In [ ]:
# Let's try the same thing with Berlin weather data!
context = """
The weather in Berlin  December of 2027 will be around 13 degrees Celsius.
Specific dates:
- 10th of December: 10 degrees Celsius
- 15th of December: 15 degrees Celsius
- 20th of December: 7 degrees Celsius
"""

message = "What will be the weather in Berlin on the 10th of December of 2027?"


generic_response = llm.call(message)
print(f"GENERIC RESPONSE:\n {generic_response}")

rag_prompt = create_rag_prompt(message=message, context=context)
rag_response = llm.call(rag_prompt)

print("-" * 30)
print(f"RAG RESPONSE:\n {rag_response}")

# Task 4
Now try it yourself! Can you find some content on the internet (think, for example, news articles or very specific, locally relevant information that the LLM normally would not have access to). 

Play around with it and let the creative juices flow. Can you discover some more use cases for which you can use RAG can help make our LLM smarter?

In [20]:

context = """
UN Climate Change News, 24 November 2024 – The UN Climate Change Conference (COP29) closed today with a new finance goal to help countries to protect their people and economies against climate disasters, and share in the vast benefits of the clean energy boom.

With a central focus on climate finance, COP29 brought together nearly 200 countries in Baku, Azerbaijan, and reached a breakthrough agreement that will:

Triple finance to developing countries, from the previous goal of USD 100 billion annually, to USD 300 billion annually by 2035.
Secure efforts of all actors to work together to scale up finance to developing countries, from public and private sources, to the amount of USD 1.3 trillion per year by 2035.
Known formally as the New Collective Quantified Goal on Climate Finance (NCQG), it was agreed after two weeks of intensive negotiations and several years of preparatory work, in a process that requires all nations to unanimously agree on every word of the agreement.

"This new finance goal is an insurance policy for humanity, amid worsening climate impacts hitting every country,” said Simon Stiell, Executive Secretary of UN Climate Change. “But like any insurance policy – it only works – if premiums are paid in full, and on time. Promises must be kept, to protect billions of lives.”

"It will keep the clean energy boom growing, helping all countries to share in its huge benefits: more jobs, stronger growth, cheaper and cleaner energy for all.”

The International Energy Agency expects global clean energy investment to exceed USD 2 trillion for the first time in 2024.
""" 

message = """
what is the main outcome of the COP 29?
""" 

generic_response = llm.call(message)
print(f"GENERIC RESPONSE:\n {generic_response}")

rag_prompt = create_rag_prompt(message=message, context=context)
rag_response = llm.call(rag_prompt)

print("-" * 30)
print(f"RAG RESPONSE:\n {rag_response}")

GENERIC RESPONSE:
 As of my last update in October 2023, the specific outcomes of COP 29 (the 29th Conference of the Parties to the United Nations Framework Convention on Climate Change) have not yet been determined, as the conference has not taken place. COP 29 is scheduled to be held in November 2023, and its outcomes will depend on the negotiations and agreements reached during the conference.

Typically, COP conferences focus on a range of issues related to climate change, including:

1. **Emission Reduction Targets**: Countries may commit to more ambitious targets for reducing greenhouse gas emissions.
2. **Financing**: Discussions on financial support for developing countries to mitigate and adapt to climate change.
3. **Technology Transfer**: Mechanisms for sharing climate-friendly technologies with developing nations.
4. **Adaptation Measures**: Strategies to help vulnerable countries adapt to the impacts of climate change.
5. **Loss and Damage**: Compensation for countries tha

# That's it! 

RAGs enrich the prompt with additional information about the topic to generate responses. The external information can come from various sources, such as PDFs, Google search results, social media posts, and more. With that, we’ve built a simple Q&A RAG.

## ===> Now head to notebook 2 on embedddings and how llms undertand language

